Decisions Under Ignorance
======================

Shane Steinert-Threlkeld

S.N.M.Steinert-Threlkeld AT uva DOT nl

A Motivating Example
------

I ride my bike to work in Amsterdam every day.  This morning, it looked rather gray, but was not raining.  Should I wear my waterproof pants and jacket or leave them at home?  If it does rain, I will stay dry, which is very nice.  If it doesn't rain, I will be too warm and will have to carry extra clothes around; not so nice.

Weighing these pros and cons, I decide to put on the waterproof clothes, just to be safe.

Some questions:
* How did I go about making that decision?
* Was it a _good_ decision?  Given my desires and values.
    - What are my desires and values?
    - Can they be measured?
    
Our primary focus will be on the second question: thus we will focus on _normative decision theory_ or _rational choice theory_.  Next time, I will say a bit about descriptive decision theory.

Basic Ingredients
-----------

* States: different possible ways the world could be, for all the decision-maker knows
    - it will rain, it won't rain

* Acts: different options that the decision-maker can choose to do
    - take waterproof clothes, leave them at home

* Outcomes: results of the choice of the decision-maker
    - dry and heavy packing, wet and light packing, dry and light packing
    
N.B.: outcomes are the _bearers of value_ for the agent.  They are determined by a state-act pair, so we will often identify them with such a pair.

In [1]:
%%HTML
<style type="text/css">
.rendered_html tbody tr td:first-child {
    border-right: 1px solid black;
}
    
.rendered_html table {
    font-size: 28px;
}
</style>

Putting the Ingredients Together
-----------

| act | rain | no rain |
| ----- | ----- | ----- |
| take clothes | dry and heavy | dry and heavy |
| leave clothes | wet and light | dry and light |

My preferences on the outcomes:
$$ DL \succ DH \succ WL$$
We can capture this with one final ingredient: a _utility_ function
$$u: S \times A \to \mathbb{R}$$
For today, however, all that matters is the _order_ of the outcomes, not their actual values.  (Terminology: the rules today only appeal to _ordinal_ utility.)

| act | rain | no rain |
| ----- | ----- | ----- |
| take clothes | 1 | 1 |
| leave clothes | 0 | 2 |

Types of Decision Problems
-----------

* Decisions Under Ignorance:
    - Multiple states of the world
    - The agent does not know which one will obtain
    - The agent _cannot_ assign probabilities to the states

* Decisions Under Risk (next time):
    - The agent can assign probabilities to the states
    - Usually uses _cardinal_ utility

NB: "Decisions under Uncertainty" is used ambiguously in the literature.

Decision Problems in Python
-----------

In [2]:
from collections import namedtuple

decision_problem = namedtuple(
    'decision_problem',
    ('states', 'actions', 'utilities', 'probabilities'),
    defaults=[None]
)

So a `decision_problem` will be an object with named fields of the appropriate kind.  `probabilities` will default to `None`.  For example:

In [3]:
ride_to_work = decision_problem(
    ('rain', 'no_rain'),
    ('take_clothes', 'leave_clothes'),
    [[1, 1], [0, 2]]
)

Choice Rules
-------

* We want to assess choices as _good_ or _bad_
* A _choice rule_ provides a criterion for so doing
* Such a rule takes as input a decision problem $D = \langle S, A, u, p \rangle$ and outputs a subset of actions, those that it deems choice-worthy.  In other words, it's a function of type $$c : D \to \mathcal{P}(A(D))$$

Dominance Avoidance
---------

Let's expand the motivating example.  I in fact have a third option: I could stay home instead of going to work for fear of getting wet.  But then I would miss an important meeting.

In [4]:
import numpy as np

ride_to_work = decision_problem(
    ('rain', 'no_rain'),
    ('take_clothes', 'leave_clothes', 'stay_home'),
    np.array([
        [1, 1],
        [0, 2],
        [-1, -1]
    ])
)

Intuitively: I should _not_ stay home because the meeting is too important.  More precisely: it's better to either take my clothes or leave them, regardless of whether it rains or not.

Dominance Avoidance
---------

We can formulate our first decision rule:

\begin{align*}
\text{avoid_dominated}(D) = \{ a \in A(D) : \lnot \exists a' \neq a \text{ s.t. }& \forall s , u(s, a') \geq u(s, a) \text{ and } \\
& \exists s , u(s, a') > u(s, a) \}
\end{align*}

(NB: this is _weak_ dominance avoidance. If we required strict inequality for all states, we would have _strong_ dominance avoidance.)

In [5]:
def avoid_dominated(decision):
    admissible = []
    for idx in range(len(decision.actions)):
        compare = decision.utilities - decision.utilities[idx]
        acts_weak = (compare >= 0).all(axis=1)
        acts_strong = (compare > 0).any(axis=1)
        dominated_by = acts_weak * acts_strong  # multiplication is like conjunction
        if not dominated_by.any():
            # dominated_by is a boolean array saying whether each action dominates the present one
            admissible.append(decision.actions[idx])
    return admissible

In [6]:
avoid_dominated(ride_to_work)

['take_clothes', 'leave_clothes']

Maximin
------

While dominance reasoning generally works, many decision problems have no actions that are dominated, and so avoidance does not help (i.e. $\text{avoid_dominated}(D) = A(D)$, so every action is admissible).

Another intuitive thought for decisions under ignorance: since you do not know what state will obtain, make sure the worst-case scenario is the best it can be.

$$\text{maximin}(D) = \text{argmax}_a \{ \text{min}_s(\{ u(s, a) \}) \}$$

Maximin
------

In [15]:
def maximin(decision):
    mins = np.amin(decision.utilities, axis=1)
    act_idxs = np.where(mins == np.amax(mins))
    return list(np.array(decision.actions)[act_idxs])  # numpy arrays have richer indexing abilities than lists :)

In [16]:
maximin(ride_to_work)

['take_clothes']